Copyright 2023 Google LLC

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at

    https://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.

# Colab to run MADLAD models

Adapted from [T5X tutorial](https://colab.research.google.com/github/google-research/t5x/blob/main/t5x/notebooks/inference.ipynb#scrollTo=f9_BPXG_QgJs)

*Special thanks to Juarez Bochi*




## Install packages

In [ ]:
!git clone https://github.com/google-research/t5x.git

Cloning into 't5x'...
remote: Enumerating objects: 6284, done.
remote: Counting objects: 100% (158/158), done.
remote: Compressing objects: 100% (90/90), done.
remote: Total 6284 (delta 82), reused 103 (delta 66), pack-reused 6126
Receiving objects: 100% (6284/6284), 9.38 MiB | 3.58 MiB/s, done.
Resolving deltas: 100% (4461/4461), done.


In [ ]:
cpu = True  # True
if cpu:
  !python -m pip uninstall -y jax jaxlib
  !python -m pip install jax[cpu]
  !cd t5x && pip install .
else:
  !cd t5x && python3 -m pip install -e '.[tpu]' -f https://storage.googleapis.com/jax-releases/libtpu_releases.html

  Using cached jax-0.4.20-py3-none-any.whl.metadata (23 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.2/61.2 kB 5.0 MB/s eta 0:00:00
  Using cached opt_einsum-3.3.0-py3-none-any.whl (65 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.4/60.4 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.8/85.8 MB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 206.7/206.7 kB 24.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 83.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 36.4/36.4 MB 62.6 MB/s eta 0:00:00
Using cached jax-0.4.20-py3-none-any.whl (1.7 MB)
Processing /usr/local/google/home/biaojiaxing/work/tmp/jupyter-server/t5x
  Preparing metadata (setup.py) ... - done
  Cloning https://github.com/google/airio to /tmp/pip-install-aawkrx82/airio_b13fee94cd1449d09f54f56a8c0c3345
  Running command git clone --filter=blob:none --quiet https://github.com/google/airio /tmp/pip-install-aawkrx82/airio_b13fee94

In [ ]:
!pip install --upgrade seqio t5 numpy==1.24.1 tensorstore==0.1.35

  Using cached seqio-0.0.18-py3-none-any.whl.metadata (51 kB)
  Using cached t5-0.9.4-py2.py3-none-any.whl (164 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 21.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.9/10.9 MB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 385.2/385.2 kB 34.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 72.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.9/118.9 kB 15.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.9/51.9 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.1/123.1 kB 15.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 840.9/840.9 kB 57.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - \ done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 341.8/341.8 kB 33.5 MB

In [ ]:

# Madlad checkpoint
!gsutil -m cp -r gs://madlad-400-checkpoints/checkpoints/3b-mt .

Copying gs://madlad-400-checkpoints/checkpoints/3b-mt/3b-mt.gin...
Copying gs://madlad-400-checkpoints/checkpoints/3b-mt/checkpoint...
Copying gs://madlad-400-checkpoints/checkpoints/3b-mt/state.param_states.decoder.decoder_norm.scale.v/0...
Copying gs://madlad-400-checkpoints/checkpoints/3b-mt/state.param_states.decoder.layers_0.pre_cross_attention_layer_norm.scale.v/0...
Copying gs://madlad-400-checkpoints/checkpoints/3b-mt/state.param_states.decoder.layers_0.pre_mlp_layer_norm.scale.v/0...
Copying gs://madlad-400-checkpoints/checkpoints/3b-mt/state.param_states.decoder.layers_0.pre_self_attention_layer_norm.scale.v/0...
Copying gs://madlad-400-checkpoints/checkpoints/3b-mt/state.param_states.decoder.layers_1.pre_cross_attention_layer_norm.scale.v/0...
Copying gs://madlad-400-checkpoints/checkpoints/3b-mt/state.param_states.decoder.layers_1.pre_mlp_layer_norm.scale.v/0...
Copying gs://madlad-400-checkpoints/checkpoints/3b-mt/state.param_states.decoder.layers_1.pre_self_attention_laye

## Rebuild .zarray files

In [ ]:
import json
import os
import orbax.checkpoint

orbax_checkpointer = orbax.checkpoint.PyTreeCheckpointer()
model = orbax_checkpointer.restore("3b-mt")

def create_jarr_files(d):
  if not isinstance(d, dict):
    return
  if "kvstore" in d:
    path = os.path.join('3b-mt', d["kvstore"]["path"], ".zarray")
    with open(path, "w") as f:
      data = d["metadata"].copy()
      data["zarr_format"] = 2
      assert d["dtype"] == "float32"
      data["dtype"] = "<f4"
      data["dimension_separator"] = "."
      data["fill_value"] = None
      data["filters"] = None
      data["order"] = "C"
      f.write(json.dumps(data))
  for v in d.values():
    create_jarr_files(v)

create_jarr_files(model)

## Set-Up

Note: If you are using the public colab (no the Pro version), you are likely run out of memory. Please consider using its `Connect to a local runtime` option by following the [setup guide](https://github.com/google-research/t5x/blob/main/t5x/notebooks/README.md).

In [ ]:

import functools

import jax
from jax.experimental import multihost_utils
import numpy as np
import os
import re
import seqio
import tensorflow as tf

from t5x.examples.t5 import network
import t5x
from t5x import models
from t5x import partitioning
from t5x import trainer as trainer_lib
from t5x import utils
from t5x.infer import _extract_tokens_and_aux_values
from t5x.interactive_model import get_dataset_from_natural_text_examples

### Model Network

In [ ]:
# t5 network
t5_config = network.T5Config(
    vocab_size=256000,
    dtype='bfloat16',
    emb_dim=1024,
    num_heads=16,
    num_encoder_layers=32,
    num_decoder_layers=32,
    head_dim=128,
    mlp_dim=8192,
    mlp_activations=('gelu', 'linear'),
    dropout_rate=0.0,
    logits_via_embedding=False)
module = network.Transformer(config=t5_config)

# vocabulary
vocabulary = seqio.SentencePieceVocabulary(
    'gs://madlad-400-checkpoints/vocabulary/256k_vocab/spm.model')

# model
model = t5x.models.EncoderDecoderModel(
    module=module,
    input_vocabulary=vocabulary,
    output_vocabulary=vocabulary,
    optimizer_def=t5x.adafactor.Adafactor(decay_rate=0.8, step_offset=0))

# downloaded checkpoint path
checkpoint_path='3b-mt'

# Misc
dtype='bfloat16'
restore_mode='specific'
# Define a partitioner.
partitioner=partitioning.PjitPartitioner(num_partitions=1)
# Define additional, miscellaneous constructor arguments.
batch_size=8
task_feature_lengths = {'inputs': 64, 'targets': 64}
output_dir='/tmp/output_dir'
input_shapes = {
    'encoder_input_tokens': np.array([8, 38]),
    'decoder_target_tokens': np.array([8, 18]),
    'decoder_input_tokens': np.array([8, 18]),
    'decoder_loss_weights': np.array([8, 18])
}

### Checkpoint Restore

In [ ]:
# 1.) Configure the Output Directory
output_dir = re.sub(r"(?<!gs:)([\/]{2,})", "/", output_dir)
if not os.path.exists(output_dir):
  os.mkdir(output_dir)

# 2.) Initialize RNGs
init_random_seed = 42
random_seed = multihost_utils.broadcast_one_to_all(np.int32(init_random_seed))
utils.set_hardware_rng_ops()
rng = jax.random.PRNGKey(random_seed)
init_rng, trainer_rng = jax.random.split(rng, 2)

# 3.) Validate the Partitioner
if partitioner._model_parallel_submesh:
  num_partitions = np.prod(partitioner._model_parallel_submesh)
else:
  num_partitions = partitioner._num_partitions
if jax.device_count() % num_partitions != 0:
  raise ValueError(
    "The number of devices available must be a multiple of the number of",
    f" partitions. There are {jax.device_count()} devices available, but",
    f" the number of partitions is set to {num_partitions}. Please",
    " provide a different number of partitions.")

# 4.) Create a Checkpoint Manager
# a.) Define CheckpointCfg wrappers.
save_checkpoint_cfg = utils.SaveCheckpointConfig(
        dtype=dtype,
        keep=5, # The number of checkpoints to keep in the output_dir.
        save_dataset=False)
restore_checkpoint_cfg = utils.RestoreCheckpointConfig(
        dtype=dtype,
        mode=restore_mode,
        path=checkpoint_path)

# b.) Define a train state initializer, which will help us get information about the
# TrainState shape.
train_state_initializer = utils.TrainStateInitializer(
        optimizer_def=model.optimizer_def,
        init_fn=model.get_initial_variables,
        input_shapes=input_shapes,
        input_types=None,
        partitioner=partitioner)

# c.) Define the checkpoint manager.
checkpoint_manager = utils.LegacyCheckpointManager(
        save_cfg=save_checkpoint_cfg,
        restore_cfg=restore_checkpoint_cfg,
        train_state_shape=train_state_initializer.global_train_state_shape,
        partitioner=partitioner,
        ds_iter=None,
        model_dir=output_dir)

### 5.) Restore the Model from a Checkpoint, or Initialize from Scratch ###
def get_state(rng):
  return train_state_initializer.from_scratch(rng).state_dict()

# a.) Try to restore a model from a checkpoint.
train_state = checkpoint_manager.restore(
  [restore_checkpoint_cfg.path],
  restore_checkpoint_cfg,
  utils.get_fallback_state(restore_checkpoint_cfg, get_state, init_rng)
)

# b.) If no checkpoint to restore, init from scratch.
if train_state is None:
  train_state = train_state_initializer.from_scratch(init_rng)

## Do the inference

Some languages supported (with parallel data)

ace
ace_Arab
af
am
an
ar
ary
arz
as
az
ba
ban
bar
be
bg
bho
bjn
bjn_Arab
bm
bn
br
bs
bug
ca
ceb
crh_Latn
cs
cy
da
de
din
dv
dz
el
en
en_xx_simple
eo
es
et
eu
fa
fi
fil
fo
fr
fr_CA
fr_ca
fur
fuv
fy
ga
gd
gl
gn
gu
ha
he
hi
hne
hr
hu
hy
id
ig
io
is
it
iu
ja
jv
ka
kk
km
kn
ko
kr
kr_Arab
ks
ks_Deva
ku
ky
la
lb
li
lij
lmo
lt
ltg
lv
mag
mg
mi
mk
ml
mn
mni
mr
ms
mt
mwl
my
nb
nds
nds_NL
nds_nl
ne
nl
nn
no
nus
oc
or
pa
pl
prs
ps
pt
pt_br
ro
ru
rw
sc
scn
sd
se
sh
shn
si
simple
sk
sl
so
sq
sr
sv
sw
szl
ta
taq
taq_Tfng
te
tg
th
tk
tl
tr
tt
tzm
ug
uk
ur
uz
vec
vi
wa
wuu
xh
yi
yo
zh
zh_Hant
zh_cn
zh_tw
zu

In [ ]:
examples = [
    '<2zh> Now both sides have renewed ambitions and have cleared the way for the visit with a series of gestures, experts say.',
    '<2es> Now both sides have renewed ambitions and have cleared the way for the visit with a series of gestures, experts say.',
    '<2de> Now both sides have renewed ambitions and have cleared the way for the visit with a series of gestures, experts say.',
    '<2en> 11月1日起，四项电动车国家标准正式实施。这些标准对电动车的设计制造、安全性能、电池系统和标识等方面都提出了明确规定，涉及电动车的车把、锂电池、总线设计、电子控制系统等。',
    '<2en> Ahora, un estudio de una universidad de Arizona ha revelado que usar redes luminosas puede resolver este problema.',
]

# tokenization
output_features = {
    "inputs":
        seqio.Feature(
            vocabulary=model.input_vocabulary, add_eos=True),
    "targets":
        seqio.Feature(
            vocabulary=model.output_vocabulary, add_eos=True)
}

# datasetm, preprocessing and feature conversion
dataset = get_dataset_from_natural_text_examples(
    examples,
    preprocessors=[
        seqio.preprocessors.tokenize,
        seqio.preprocessors.append_eos
    ],
    task_feature_lengths=task_feature_lengths,
    features=output_features)
feature_converter = model.FEATURE_CONVERTER_CLS(pack=False)
model_dataset = feature_converter(
    dataset, task_feature_lengths=task_feature_lengths)

# inference function
infer_fn = functools.partial(
  utils.get_infer_fn(
    infer_step=model.predict_batch_with_aux,
    batch_size=batch_size,
    train_state_axes=train_state_initializer.train_state_axes,
    partitioner=partitioner),
  train_state=train_state)

In [ ]:
# Infererence and get the predictions
predictions, aux_values = _extract_tokens_and_aux_values(
    infer_fn(model_dataset.enumerate()))

inferences = []
for idx, inputs in model_dataset.enumerate().as_numpy_iterator():
  prediction = np.asarray(predictions[idx]).astype(np.int32)
  source = np.asarray(inputs['encoder_input_tokens']).astype(np.int32)

  # Remove padding values
  source = source[source != 0]
  prediction = prediction[prediction != 0]

  # Decode into string
  prediction = output_features["targets"].vocabulary.decode_tf(
      tf.constant(prediction)).numpy()
  source = output_features["inputs"].vocabulary.decode_tf(
      tf.constant(source)).numpy()
  inferences.append((source, prediction, aux_values['scores'][idx]))

for src, hyp, score in inferences:
  print(f"Source: {src.decode('utf-8')}")
  print(f"Translation: {hyp.decode('utf-8')}")
  print(f"Score: {score}")
  print()


normalizer.cc(51) LOG(INFO) precompiled_charsmap is empty. use identity normalization.
normalizer.cc(51) LOG(INFO) precompiled_charsmap is empty. use identity normalization.


Source: <2zh> Now both sides have renewed ambitions and have cleared the way for the visit with a series of gestures, experts say.
Translation: 专家们认为,双方都重新提出了雄心壮志,并通过一系列姿态为访问扫清了道路。
Score: -5.694250106811523

Source: <2es> Now both sides have renewed ambitions and have cleared the way for the visit with a series of gestures, experts say.
Translation: Ahora ambas partes han renovado sus ambiciones y han despejado el camino para la visita con una serie de gestos, dicen los expertos.
Score: -2.3198225498199463

Source: <2de> Now both sides have renewed ambitions and have cleared the way for the visit with a series of gestures, experts say.
Translation: Jetzt haben beide Seiten ihre Ambitionen erneuert und den Weg für den Besuch mit einer Reihe von Gesten frei gemacht, sagen Experten.
Score: -3.43707537651062

Source: <2en> 11月1日起，四项电动车国家标准正式实施。这些标准对电动车的设计制造、安全性能、电池系统和标识等方面都提出了明确规定，涉及电动车的车
Translation: As of 1 November, four national standards for electric vehicles have been officially impl

normalizer.cc(51) LOG(INFO) precompiled_charsmap is empty. use identity normalization.
